In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from skimage import color
from skimage import io
import matplotlib.cm as cm
from scipy.misc import imresize
from scipy.ndimage.filters import gaussian_filter
import cv2

C:\Anaconda-python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../MNIST_datxa/", one_hot=True)

Extracting ../MNIST_datxa/train-images-idx3-ubyte.gz
Extracting ../MNIST_datxa/train-labels-idx1-ubyte.gz
Extracting ../MNIST_datxa/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_datxa/t10k-labels-idx1-ubyte.gz


In [3]:
#make data path to list
hnd_train = glob('../../Datas/letter/Hnd/Digit/train/*/*')
hnd_test = glob('../../Datas/letter/Hnd/Digit/test/*/*')
real_train = glob('../../Datas/letter/EnglishImg/English/Img/GoodImg/English_image/Digit/train/*/*')
real_test = glob('../../Datas/letter/EnglishImg/English/Img/GoodImg/English_image/Digit/test/*/*')

In [7]:
#get label from the path
def get_label_from_path(path):
    return (int(path.split('\\')[-2]))

#Read images and pre-processing

def read_image(path):
    img = cv2.imread(path) # read the image from the path
    img = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # make gray
    gray_mean = np.mean(gray)
    ret,thresh = cv2.threshold(gray,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )  #make binary
    kernel = np.ones((1,1), np.uint8) 
    img_dilation = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel) #to get more clear image
    im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #find contours
    x, y, w, h = cv2.boundingRect(ctrs[0]) # make rectangle following out line
    if h > w:
        x -= (h - w)//2
        w += (h - w)
    elif h < w:
        y -= (w - h)//2
        h += (w - h)
    roi = img_dilation[y:y+h, x:x+w]
    roi_resized = imresize(roi, (28, 28)).reshape(784)
    roi_resized = np.array(roi_resized, dtype= np.float32)
    for i in range(len(roi_resized)):
        roi_resized[i] /= 255
    return roi_resized

#Read the images don't need to ROI,
def read_image_not_roi(path):
    img = cv2.imread(path) # read the image from the path
    img = cv2.GaussianBlur(img, (5, 5), 0)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # make gray
    gray_mean = np.mean(gray)
    ret,thresh = cv2.threshold(gray,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )  #make binary
    kernel = np.ones((1,1), np.uint8) 
    img_dilation = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel) #to get more clear image
    unique, counts = np.unique(img_dilation, return_counts = True)
    num_dict = dict(zip(unique, counts))
    if(num_dict[255] >= num_dict[0]):
        ret,img_dilation = cv2.threshold(img_dilation,gray_mean,255,cv2.THRESH_BINARY_INV +cv2.THRESH_OTSU )
    image = imresize(img_dilation, (28, 28)).reshape(784)
    image = np.array(image, dtype= np.float32)
    for i in range(len(image)):
        image[i] /= 255
    return image

#make label as a list
path = hnd_train[2]
path, get_label_from_path(path)
class_name = get_label_from_path(path)
label_name_list_train = []

for path in hnd_train:
    label_name_list_train.append(get_label_from_path(path))

    
unique_label_names = np.unique(label_name_list_train)

#onehot encoding
def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path(path)
    onehot_label = onehot_label.astype(np.uint8)
    return onehot_label

In [8]:
onehot_encode_label(path)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [9]:
#Unite the images to one dataset

def make_dataset(image_shape, label_shape, path_images):
    initial_image_array = np.zeros([len(path_images),image_shape])
    initial_label_array = np.zeros([len(path_images),label_shape])
    for n, path in enumerate(path_images):
        image = read_image(path)
        onehot_label = onehot_encode_label(path)
        initial_image_array[n, :] = image
        initial_label_array[n, :] = onehot_label
    return initial_image_array, initial_label_array

def make_dataset_real(image_shape, label_shape, path_images):
    initial_image_array = np.zeros([len(path_images),image_shape])
    initial_label_array = np.zeros([len(path_images),label_shape])
    for n, path in enumerate(path_images):
        image = read_image_not_roi(path)
        onehot_label = onehot_encode_label(path)
        initial_image_array[n, :] = image
        initial_label_array[n, :] = onehot_label
    return initial_image_array, initial_label_array

In [10]:
data_train_images_hnd, data_train_labels_hnd = make_dataset(784, 10, hnd_train)
data_test_images_hnd, data_test_labels_hnd = make_dataset(784, 10, hnd_test)

C:\Anaconda-python\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [11]:
data_test_images_real, data_test_labels_real = make_dataset_real(784, 10, real_test)
data_train_images_real, data_train_labels_real =make_dataset_real(784, 10, real_train)

C:\Anaconda-python\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


# The model trained with hand writing dataset

In [20]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100


In [28]:
#hand writing dataset 으로만 학습시킨 데이터

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('hand write data training start')
for i in range(len(hnd_train)):
    feed_dict = {X: data_train_images_hnd, Y: data_train_labels_hnd, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('step: {0}, cost: {1}'.format(i, c))
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))

hand write data training start
step: 0, cost: 3.950756549835205
step: 100, cost: 0.036300428211688995
step: 200, cost: 0.0060258349403738976
step: 300, cost: 0.0169266015291214
step: 400, cost: 0.0016593559412285686
Learning Finished!
Accuracy: 0.96363634


In [29]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


Accuracy: 0.2102


In [30]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))


Accuracy: 0.4391892


# The model trained with mnist dataset

In [15]:
#mnist dataset만으로 학습시킨 데이터

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

# Get one and predict
r = np.random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1], keep_prob: 1}))


Epoch: 0001 cost = 0.656438541
Epoch: 0002 cost = 0.236185142
Epoch: 0003 cost = 0.171984848
Epoch: 0004 cost = 0.139384079
Epoch: 0005 cost = 0.119217401
Epoch: 0006 cost = 0.107606449
Epoch: 0007 cost = 0.094643766
Epoch: 0008 cost = 0.088591053
Epoch: 0009 cost = 0.078673688
Epoch: 0010 cost = 0.071121083
Epoch: 0011 cost = 0.067316908
Epoch: 0012 cost = 0.062557119
Epoch: 0013 cost = 0.061370156
Epoch: 0014 cost = 0.055523415
Epoch: 0015 cost = 0.051291360
Learning Finished!
Accuracy: 0.9804
Label:  [9]
Prediction:  [9]


In [16]:
#test hand writing data and real images data

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))


Accuracy: 0.4


In [17]:

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))


Accuracy: 0.3783784


# The model trained with real image dataset

In [32]:
#real image dataset 으로 학습시킨 데이터

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(len(real_train)):
    feed_dict = {X: data_train_images_real, Y: data_train_labels_real, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('step: {0}, cost: {1}'.format(i, c))
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))


step: 0, cost: 4.48777437210083
step: 100, cost: 0.34043088555336
step: 200, cost: 0.021645277738571167
step: 300, cost: 0.015236204490065575
step: 400, cost: 0.005572722293436527
Learning Finished!
Accuracy: 0.8243243


In [33]:
#mnist 와 hand writing data 로 학습데이터 테스트
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


Accuracy: 0.23


In [34]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))


Accuracy: 0.6181818


# The model trained with all dataset

In [21]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

for i in range(len(hnd_train)):
    feed_dict = {X: data_train_images_hnd, Y: data_train_labels_hnd, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('hand writing step: {0}, cost: {1}'.format(i, c))
        
for i in range(len(real_train)):
    feed_dict = {X: data_train_images_real, Y: data_train_labels_real, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('real image step: {0}, cost: {1}'.format(i, c))
        
        
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))



Epoch: 0001 cost = 0.675410449
Epoch: 0002 cost = 0.228526456
Epoch: 0003 cost = 0.173301365
Epoch: 0004 cost = 0.137239344
Epoch: 0005 cost = 0.119886233
Epoch: 0006 cost = 0.103147588
Epoch: 0007 cost = 0.091768273
Epoch: 0008 cost = 0.080930068
Epoch: 0009 cost = 0.077476237
Epoch: 0010 cost = 0.072503127
Epoch: 0011 cost = 0.067268184
Epoch: 0012 cost = 0.058945879
Epoch: 0013 cost = 0.058078160
Epoch: 0014 cost = 0.055456169
Epoch: 0015 cost = 0.050322427
hand writing step: 0, cost: 7.300313949584961
hand writing step: 100, cost: 0.0105508454144001
hand writing step: 200, cost: 0.006833203602582216
hand writing step: 300, cost: 0.002494679531082511
hand writing step: 400, cost: 0.0004339423030614853
real image step: 0, cost: 12.111218452453613
real image step: 100, cost: 0.1381896585226059
real image step: 200, cost: 0.06627186387777328
real image step: 300, cost: 0.006897913292050362
real image step: 400, cost: 0.018273666501045227
Learning Finished!
Accuracy: 0.7215
Accuracy: 0.

# The model trained with mnist, hand dateset

In [22]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

for i in range(len(hnd_train)):
    feed_dict = {X: data_train_images_hnd, Y: data_train_labels_hnd, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('hand writing step: {0}, cost: {1}'.format(i, c))
        
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))



Epoch: 0001 cost = 0.625789662
Epoch: 0002 cost = 0.222770260
Epoch: 0003 cost = 0.167252021
Epoch: 0004 cost = 0.136080719
Epoch: 0005 cost = 0.116599096
Epoch: 0006 cost = 0.102926976
Epoch: 0007 cost = 0.089039868
Epoch: 0008 cost = 0.084224086
Epoch: 0009 cost = 0.072579885
Epoch: 0010 cost = 0.068545061
Epoch: 0011 cost = 0.060173794
Epoch: 0012 cost = 0.058822827
Epoch: 0013 cost = 0.056794250
Epoch: 0014 cost = 0.052918267
Epoch: 0015 cost = 0.050963464
hand writing step: 0, cost: 6.069091320037842
hand writing step: 100, cost: 0.01204571034759283
hand writing step: 200, cost: 0.0018487004563212395
hand writing step: 300, cost: 0.005085427779704332
hand writing step: 400, cost: 0.0028450593817979097
Learning Finished!
Accuracy: 0.7583
Accuracy: 0.93636364


# The model trained with mnist, real dataset

In [23]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        
for i in range(len(real_train)):
    feed_dict = {X: data_train_images_real, Y: data_train_labels_real, keep_prob:0.8}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('real image step: {0}, cost: {1}'.format(i, c))
        
        
print('Learning Finished!')

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))



Epoch: 0001 cost = 0.669421206
Epoch: 0002 cost = 0.231278737
Epoch: 0003 cost = 0.168394644
Epoch: 0004 cost = 0.140194413
Epoch: 0005 cost = 0.114275611
Epoch: 0006 cost = 0.101400622
Epoch: 0007 cost = 0.091687169
Epoch: 0008 cost = 0.079398917
Epoch: 0009 cost = 0.075927305
Epoch: 0010 cost = 0.066257798
Epoch: 0011 cost = 0.065783447
Epoch: 0012 cost = 0.062731737
Epoch: 0013 cost = 0.055429863
Epoch: 0014 cost = 0.053476608
Epoch: 0015 cost = 0.050976418
real image step: 0, cost: 8.866402626037598
real image step: 100, cost: 0.017531603574752808
real image step: 200, cost: 0.007219364400953054
real image step: 300, cost: 0.003533955430611968
real image step: 400, cost: 0.003851620713248849
Learning Finished!
Accuracy: 0.852
Accuracy: 0.8581081


# The model trained with real, hand dataset

In [38]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.int8, [None, 10])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape= [784, 512],
                     initializer= tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)

W2 = tf.get_variable("W2", shape= [512, 256],
                     initializer= tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob= keep_prob)

W3 = tf.get_variable("W3", shape= [256, 128],
                     initializer= tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob= keep_prob)

W4 = tf.get_variable("W4" ,shape= [128, 64],
                     initializer= tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob= keep_prob)

W5 = tf.get_variable("W5" ,shape= [64, 10],
                     initializer= tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for i in range(len(hnd_train)):
    feed_dict = {X: data_train_images_hnd, Y: data_train_labels_hnd, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('hand writing step: {0}, cost: {1}'.format(i, c))
        
for i in range(len(real_train)):
    feed_dict = {X: data_train_images_real, Y: data_train_labels_real, keep_prob:0.7}
    c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    if i % 100 == 0:
        print('real image step: {0}, cost: {1}'.format(i, c))
        
        
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_hnd, Y: data_test_labels_hnd, keep_prob: 1}))

correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: data_test_images_real, Y: data_test_labels_real, keep_prob: 1}))



hand writing step: 0, cost: 4.675471782684326
hand writing step: 100, cost: 0.03708893060684204
hand writing step: 200, cost: 0.007986089214682579
hand writing step: 300, cost: 0.0028995703905820847
hand writing step: 400, cost: 0.0028997340705245733
real image step: 0, cost: 12.022271156311035
real image step: 100, cost: 0.11180675029754639
real image step: 200, cost: 0.039071545004844666
real image step: 300, cost: 0.014362362213432789
real image step: 400, cost: 0.008015769533813
Accuracy: 0.7181818
Accuracy: 0.8445946
